In [8]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
data_skripsi=pd.read_csv('FIX_DATASKRIPSI_NEW.CSV', sep=';', encoding= 'unicode_escape')
df = data_skripsi.pivot_table(index='Item_name', columns='User_Name', values='Rating').fillna(0)
df1 = df.copy()
df1

User_Name,A,A AGUSLIMI SHAFIRA PUTRI AP,A Dian Christianti,AA,AAN AMBARYATI,AB,ABDUL WAHAB ISKANDAR,AC,ACHMAT LUTHFI YAKIM,AD,...,yuri yah,yusuf satriyo,yuyun sutaji,yuzon erman,zahra azzahra,zainul arifin,zanni erdiawan saputra,zevina Liemahassana,ziah nur aisjah,zuraidah fitri lubis
Item_name,,,,,,,,,,,,,,,,,,,,,
@HOM Premiere Timoho,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARTOTEL Suites Bianti - Yogyakarta,0.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abadi Hotel Malioboro Yogyakarta by Tritama Hospitality,5.0,0.0,0.0,3.6,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Adhisthana Hotel,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Airlangga Hotel,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Votel Maerakatja Yogyakarta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YATS Colony,0.0,0.0,0.0,3.2,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yellow Star Ambarrukmo Hotel,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
user_index = df1.columns.tolist()

In [27]:
def get_user_rating(df, user):
  return df[df[user] > 0][user].index.tolist()

def recommend_hotels(df, df1, user, num_recommended_hotel):
  recommended_hotel = {}
  for m in df[df[user] == 0].index.tolist():
    index_df = df.index.tolist().index(m)
    predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
    recommended_hotel[m] = predicted_rating

  sorted_rm = {k: v for k, v in sorted(recommended_hotel.items(), 
      key=lambda item: item[1], reverse=True)[:num_recommended_hotel]}
  return sorted_rm

In [11]:
def hotels_recommender(df, user, num_neighbors, num_recommendation):
  df1 = df.copy()

  number_neighbors = num_neighbors

  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(df.values)
  distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

  user_index = df.columns.tolist().index(user)

  for m,t in list(enumerate(df.index)):
    if df.iloc[m, user_index] == 0:
      sim_hotels = indices[m].tolist()
      hotels_distances = distances[m].tolist()
    
      if m in sim_hotels:
        id_hotels = sim_hotels.index(m)
        sim_hotels.remove(m)
        hotels_distances.pop(id_hotels) 

      else:
        sim_hotels = sim_hotels[:num_neighbors-1]
        hotels_distances = hotels_distances[:num_neighbors-1]
            
      hotels_similarity = [1-x for x in hotels_distances]
      hotels_similarity_copy = hotels_similarity.copy()
      nominator = 0

      for s in range(0, len(hotels_similarity)):
        if df.iloc[sim_hotels[s], user_index] == 0:
          if len(hotels_similarity_copy) == (number_neighbors - 1):
            hotels_similarity_copy.pop(s)
          
          else:
            hotels_similarity_copy.pop(s-(len(hotels_similarity)-len(hotels_similarity_copy)))
            
        else:
          nominator = nominator + hotels_similarity[s]*df.iloc[sim_hotels[s],user_index]
          
      if len(hotels_similarity_copy) > 0:
        if sum(hotels_similarity_copy) > 0:
          predicted_r = nominator/sum(hotels_similarity_copy)
        
        else:
          predicted_r = 0

      else:
        predicted_r = 0
        
      df1.iloc[m,user_index] = predicted_r

  return recommend_hotels(df, df1, user,num_recommendation)

In [28]:
hotels_recommender(df, 'Afni Manopo', 5, 10)

{'Adhisthana Hotel ': 4.0,
 'Brothers Inn Babarsari': 4.0,
 'Forriz Hotel Yogyakarta': 4.0,
 'Grand Kangen Hotel Urip Sumoharjo Yogyakarta': 4.0,
 'Hotel Bladok & Restaurant': 4.0,
 'Ibis Yogyakarta Adi Sucipto': 4.0,
 'Jambuluwuk Malioboro Hotel Yogyakarta': 4.0,
 'Kalya Hotel Yogyakarta': 4.0,
 'Kotta GO Yogyakarta': 4.0,
 'Lafayette Boutique Hotel': 4.0}

In [37]:
hotels = data_skripsi[["Item_id", "Item_name", "Bintang_Hotel"]].drop_duplicates().reset_index(drop=True)

In [38]:
hotels.to_csv("data_hotels.csv", index=False)

In [41]:
def batch()

,No,User_id,User_Name,Item_id,Item_name,Rating,PostDate,Bintang_Hotel
0,1,2700,Hendra Hendra,1,Eastparc Hotel Yogyakarta,4.0,08/07/2020,5
1,2,832,AS,1,Eastparc Hotel Yogyakarta,4.2,30/08/2020,5
2,3,4380,NU,1,Eastparc Hotel Yogyakarta,5.0,12/02/2022,5
3,4,3483,Leny Wati,1,Eastparc Hotel Yogyakarta,5.0,11/01/2022,5
4,5,4116,Mujiburahman,1,Eastparc Hotel Yogyakarta,5.0,11/12/2021,5
5,6,1244,Christina Setiawan,1,Eastparc Hotel Yogyakarta,5.0,09/12/2021,5
6,7,1773,DV,1,Eastparc Hotel Yogyakarta,5.0,05/12/2021,5
7,8,6121,Vivi Chai,1,Eastparc Hotel Yogyakarta,5.0,26/10/2021,5
8,9,6383,YI,1,Eastparc Hotel Yogyakarta,5.0,15/07/2021,5
9,10,2940,Imam Nasuha,1,Eastparc Hotel Yogyakarta,5.0,03/05/2021,5


In [44]:
data_skripsi.shape

(12527, 8)

In [47]:
round(12527/15)

835

In [54]:
def batch(df, n):
    long = round(df.shape[0]/n)
    last = 0
    for i in range(n):
        if i == n-1:
            df.iloc[last:].to_csv("ratings_{}.csv".format(i), index=False)
        elif i==0:
            df.iloc[:long].to_csv("ratings_{}.csv".format(i), index=False)
        else:
            until = last + long
            df.iloc[last:until].to_csv("ratings_{}.csv".format(i), index=False)
        
        last = last + long

In [55]:
batch(data_skripsi, 15)

In [51]:
[ i for i in range(5)]

[0, 1, 2, 3, 4]